In [1]:
#this program predict stock prices by using Machine Leearning models

import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [2]:
df = quandl.get("WIKI/FB")
print(df.head(5))

             Open   High    Low    Close       Volume  Ex-Dividend  \
Date                                                                 
2012-05-18  42.05  45.00  38.00  38.2318  573576400.0          0.0   
2012-05-21  36.53  36.66  33.00  34.0300  168192700.0          0.0   
2012-05-22  32.61  33.59  30.94  31.0000  101786600.0          0.0   
2012-05-23  31.37  32.50  31.36  32.0000   73600000.0          0.0   
2012-05-24  32.95  33.21  31.77  33.0300   50237200.0          0.0   

            Split Ratio  Adj. Open  Adj. High  Adj. Low  Adj. Close  \
Date                                                                  
2012-05-18          1.0      42.05      45.00     38.00     38.2318   
2012-05-21          1.0      36.53      36.66     33.00     34.0300   
2012-05-22          1.0      32.61      33.59     30.94     31.0000   
2012-05-23          1.0      31.37      32.50     31.36     32.0000   
2012-05-24          1.0      32.95      33.21     31.77     33.0300   

           

In [3]:
df = df[['Adj. Close']]
df

,Adj. Close
Date,
2012-05-18,38.2318
2012-05-21,34.0300
2012-05-22,31.0000
2012-05-23,32.0000
2012-05-24,33.0300
...,...
2018-03-21,169.3900
2018-03-22,164.8900
2018-03-23,159.3900


In [4]:
# A variable for predicting 'n' days out into the future
forecast_out = 30
#create another columns ( the target or dependent variable) shifted 'n' units up
df['Prediction'] = df[['Adj. Close']].shift(-forecast_out)
df.head(5)

,Adj. Close,Prediction
Date,,
2012-05-18,38.2318,30.771
2012-05-21,34.0300,31.200
2012-05-22,31.0000,31.470
2012-05-23,32.0000,31.730
2012-05-24,33.0300,32.170


In [5]:
df.tail()

,Adj. Close,Prediction
Date,,
2018-03-21,169.39,NaN
2018-03-22,164.89,NaN
2018-03-23,159.39,NaN
2018-03-26,160.06,NaN
2018-03-27,152.19,NaN


In [6]:
# create independent dataset
#convert dataframe to a numpya array
X = np.array(df.drop(['Prediction'],1))
#remove the list 'n' rows
X = X[:-forecast_out]
print(X)

[[ 38.2318]
 [ 34.03  ]
 [ 31.    ]
 ...
 [171.5499]
 [175.98  ]
 [176.41  ]]


C:\Users\mehak\AppData\Local\Temp/ipykernel_14180/2571028453.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = np.array(df.drop(['Prediction'],1))


In [7]:
# create dependent dataset(y)
#convert datframe to a numpy array(all valuse including the NaN's)
y = np.array(df['Prediction'])
#get all of the y valuse except the last 'n' rows
y = y[:-forecast_out]
y

array([ 30.771,  31.2  ,  31.47 , ..., 159.39 , 160.06 , 152.19 ])

In [8]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2)

In [9]:
# create and train the support vector machine(regressor)
svr_rbf = SVR(kernel='rbf',C=1e3,gamma=.1)
svr_rbf.fit(X_train,y_train)

SVR(C=1000.0, gamma=0.1)

In [10]:
#testing model
svm_confidence = svr_rbf.score(X_test,y_test)
print("svm confidence : ", svm_confidence)

svm confidence :  0.9832814933099251


In [11]:
# create and train the linear regression model
lr =LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [12]:
#testing model
lr_confidence = lr.score(X_test,y_test)
print("lr confidence : ", lr_confidence)

lr confidence :  0.9798912246785152


In [13]:
#set X_forcast equal to last 30 rows of the original data set Adj. Close column
X_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
X_forecast

C:\Users\mehak\AppData\Local\Temp/ipykernel_14180/2475898302.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]


array([[173.15],
       [179.52],
       [179.96],
       [177.36],
       [176.01],
       [177.91],
       [178.99],
       [183.29],
       [184.93],
       [181.46],
       [178.32],
       [175.94],
       [176.62],
       [180.4 ],
       [179.78],
       [183.71],
       [182.34],
       [185.23],
       [184.76],
       [181.88],
       [184.19],
       [183.86],
       [185.09],
       [172.56],
       [168.15],
       [169.39],
       [164.89],
       [159.39],
       [160.06],
       [152.19]])

In [14]:
#print linear regression model predictions for next 'n days'
lr_prediction = lr.predict(X_forecast)
lr_prediction

array([177.14927165, 183.58888329, 184.03369163, 181.40527871,
       180.04052586, 181.96128914, 183.05309143, 187.40008201,
       189.05800401, 185.55008369, 182.37576964, 179.96976089,
       180.65719196, 184.47849997, 183.85172458, 187.82467179,
       186.43970037, 189.36128242, 188.88614624, 185.97467347,
       188.30991725, 187.976311  , 189.21975249, 176.55282411,
       172.09463143, 173.34818221, 168.79900601, 163.23890177,
       163.91622356, 155.96021986])

In [15]:
#print support vector machine predictions for next 'n days'
svm_prediction = svr_rbf.predict(X_forecast)
svm_prediction

array([178.88889553, 181.31771937, 181.33425671, 178.84659658,
       174.95986357, 180.31900708, 181.44204253, 185.81597673,
       179.46829979, 185.2017892 , 181.05008667, 174.82524487,
       176.53182313, 181.80928756, 181.28864271, 183.79925813,
       187.62115405, 179.45696312, 179.68363111, 186.69672841,
       181.46658866, 183.03628549, 179.4057748 , 179.21599249,
       171.18434398, 172.58199736, 172.24863881, 167.72854428,
       166.23373089, 162.10522719])